In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Cargar el dataset
file_path = '../data/processed/df_cancer_prostata_processed.csv'
df = pd.read_csv(file_path, sep=';')

# Identificar características y variable objetivo
X = df.drop(columns=['SOBREVIVE'])
y = df['SOBREVIVE']

# Identificar variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocesamiento de las características
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# División del dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creación y entrenamiento del modelo
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=42))])

clf.fit(X_train, y_train)

# Predicciones y evaluación del modelo
y_pred = clf.predict(X_test)

# Reporte de clasificación y matriz de confusión
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(report)
print(conf_matrix)


              precision    recall  f1-score   support

         0.0       1.00      0.91      0.95        99
         1.0       0.98      1.00      0.99       430

    accuracy                           0.98       529
   macro avg       0.99      0.95      0.97       529
weighted avg       0.98      0.98      0.98       529

[[ 90   9]
 [  0 430]]


### Resultados del Modelo de Predicción

### Reporte de Clasificación

Precisión: 1.00 para la clase 0 (no sobrevive) y 0.99 para la clase 1 (sobrevive).

Recall: 0.94 para la clase 0 y 1.00 para la clase 1.

F1-Score: 0.97 para la clase 0 y 0.99 para la clase 1.

Exactitud Global: 0.99

Matriz de Confusión

Clase 0 (no sobrevive): 90 verdaderos negativos y 9 falsos positivos.

Clase 1 (sobrevive): 0 falsos negativos y 430 verdaderos positivos.

El modelo Random Forest muestra un rendimiento excelente con una precisión y recall casi perfectos. Esto indica que el modelo es muy bueno prediciendo la supervivencia de los pacientes con cáncer de próstata en este dataset.



In [4]:
from sklearn.model_selection import cross_val_score

# Validación cruzada con 5 folds
cv_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')

# Mostrar los resultados de la validación cruzada
cv_scores.mean(), cv_scores.std()


(0.9557491264249298, 0.010325122725763775)

Resultados de la Validación Cruzada

Precisión media: 0.955

Desviación estándar: 0.010

Estos resultados muestran que el modelo mantiene un rendimiento consistentemente alto en diferentes divisiones del dataset, lo cual indica una buena generalización.

In [9]:
from sklearn.model_selection import GridSearchCV
import seaborn as sns
# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ejecutar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y el mejor score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Guardar el modelo entrenado
import joblib
joblib.dump(grid_search.best_estimator_, '../models/cancer_survival_model-RandomForestClassifier.pkl')

['../models/cancer_survival_model-RandomForestClassifier.pkl']

In [8]:
from tabulate import tabulate

# Convertir best_params en una lista de listas para tabular
params_table = [[key, value] for key, value in best_params.items()]

# Imprimir best_params con tabulate
print(tabulate(params_table, headers=["Hiperparámetro", "Mejor Valor"]))

# Mostrar el mejor score
print(f"\nMejor Score: {best_score}")

Hiperparámetro                   Mejor Valor
-----------------------------  -------------
classifier__max_depth                     30
classifier__min_samples_leaf               1
classifier__min_samples_split              2
classifier__n_estimators                 100

Mejor Score: 0.9825059101654846


In [15]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el modelo guardado
modelo = joblib.load('../models/cancer_survival_model-RandomForestClassifier.pkl')

# Obtener la importancia de las características
importancias = modelo.named_steps['classifier'].feature_importances_

# Codificar y escalar características para obtener nombres de características transformadas
preprocessor = modelo.named_steps['preprocessor']
preprocessor.fit(X)

# Obtener los nombres de las columnas transformadas
onehot_columns = preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(categorical_features)
feature_names = numeric_features.tolist() + onehot_columns.tolist()

# Verificar la longitud de feature_names e importancias
print(f'Número de características: {len(feature_names)}')
print(f'Número de importancias: {len(importancias)}')

# Asegurarse de que las longitudes coincidan
if len(feature_names) == len(importancias):
    # Crear un DataFrame para visualizar las importancias
    importancia_df = pd.DataFrame({
        'Característica': feature_names,
        'Importancia': importancias
    })

    # Ordenar por importancia
    importancia_df = importancia_df.sort_values(by='Importancia', ascending=False).reset_index(drop=True)

    # Visualizar las importancias
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importancia', y='Característica', data=importancia_df.head(20))
    plt.title('Importancia de las Características (Top 20)')
    plt.show()
else:
    print("Error: Las longitudes de 'feature_names' e 'importancias' no coinciden.")




Número de características: 2642
Número de importancias: 2230
Error: Las longitudes de 'feature_names' e 'importancias' no coinciden.


In [16]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# Cargar datos
file_path = '../data/processed/df_cancer_prostata_processed.csv'  # Actualiza la ruta del archivo
df = pd.read_csv(file_path, delimiter=';')

# Identificar características y variable objetivo
X = df.drop(columns=['SOBREVIVE'])
y = df['SOBREVIVE']

# Identificar variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocesamiento de las características
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# División del dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar manualmente los hiperparámetros del modelo Random Forest
clf_manual = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', RandomForestClassifier(
                                 n_estimators=200,
                                 max_depth=20,
                                 min_samples_split=5,
                                 min_samples_leaf=2,
                                 random_state=42))])

# Entrenar el modelo
clf_manual.fit(X_train, y_train)

# Guardar el modelo entrenado
joblib.dump(clf_manual, '../models/cancer_survival_model-RandomForestClassifier.pkl')

# Cargar el modelo guardado
modelo = joblib.load('../models/cancer_survival_model-RandomForestClassifier.pkl')

# Obtener la importancia de las características
importancias = modelo.named_steps['classifier'].feature_importances_

# Codificar y escalar características para obtener nombres de características transformadas
preprocessor = modelo.named_steps['preprocessor']
preprocessor.fit(X)

# Obtener los nombres de las columnas transformadas
onehot_columns = preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(categorical_features)
feature_names = numeric_features.tolist() + onehot_columns.tolist()

# Verificar la longitud de feature_names e importancias
print(f'Número de características: {len(feature_names)}')
print(f'Número de importancias: {len(importancias)}')

# Mostrar algunas características y sus importancias para diagnosticar
print("Características y sus importancias:")
for name, importance in zip(feature_names[:10], importancias[:10]):
    print(f"{name}: {importance}")

# Si las longitudes no coinciden, imprimir las longitudes exactas
if len(feature_names) != len(importancias):
    print(f"Longitud de feature_names: {len(feature_names)}")
    print(f"Longitud de importancias: {len(importancias)}")
else:
    # Crear un DataFrame para visualizar las importancias
    importancia_df = pd.DataFrame({
        'Característica': feature_names,
        'Importancia': importancias
    })

    # Ordenar por importancia
    importancia_df = importancia_df.sort_values(by='Importancia', ascending=False).reset_index(drop=True)

    # Visualizar las importancias
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importancia', y='Característica', data=importancia_df.head(20))
    plt.title('Importancia de las Características (Top 20)')
    plt.show()


Número de características: 2642
Número de importancias: 2230
Características y sus importancias:
COD_TOPOLOGIA: 0.0
DIAS_HASTA_INICIO_TRATAMIENTO: 0.019809539491746835
DIAS_DESDE_NACIMIENTO_A_DIAGNO: 0.020172654608012118
DIAS_DESDE_INGRESO_A_DIAGNO: 0.012202379107943636
DIAS_DESDE_DIAGNO_A_COMITE: 0.010118934635609525
DIAS_DESDE_TOM_MUESTRA_A_DIAGNO: 0.00869965034599194
DIAS_DESDE_DIAGNO_TRATAMIENTO_2: 0.010021698144805953
DIAS_DESDE_TRATAMIENTO_1_A_TRATAMIENTO_2: 0.013368726848596116
DIAS_TRATAMIENTO_1: 0.0213816545314099
DIAS_TRATAMIENTO_2: 0.01143864846517943
Longitud de feature_names: 2642
Longitud de importancias: 2230
